In [176]:
import pandas as pd
so_file='fwso/sample txtfile.txt'
colspecs = [(0, 2), (20, 30), (31, 59), (60, 160),(161,164),(165,194),(195,197),(198,205),(206,235),(236,288),(288,290),(290,292),(361,364),(364,377),(394,460)]
update=pd.read_fwf(so_file,
                   header=None, 
                   colspecs=colspecs,
                   names=['dir_code','mem_wstd','first_name','last_name','sam_city','sam_stname','sam_stsubt','sam_stnmfr','bla','ble','listcode','acc_type','regulatory_id','acct_no','dist_code'],
                   converters = {'mem_wstd': str, 'sam_stnmfr':str,'acct_no':str},
                   index_col=None
                  )
#update.()
update = update.fillna('')

update.sort_values(by='last_name',inplace=True)
update.reset_index(drop=True,inplace=True)
update['acc_type']=update['acc_type'].astype('category')
update['src']=so_file
update

,dir_code,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,bla,ble,listcode,acc_type,regulatory_id,acct_no,dist_code,src
0,BL,002236040,ARLEEN GARCIA,CRISTOBAL,OBA,TAWIRAN,RD,45,,,PB,RR,PLD,0241336611,BULACAN OBANDO,fwso/sample txtfile.txt
1,BL,002236059,,JOCELYN FORGE INC,MEY,BAGBAGUIN,RD,56,,,PB,BR,PLD,0161375209,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt
2,BL,002291961,,L.G RAMOS TRADING AND MARINESERVICES,OBA,F NAVARETTE,ST,25,,,PB,BR,PLD,0212837784,BULACAN OBANDO,fwso/sample txtfile.txt
3,BL,002236049,ARRIANE ARGAMOSA,LAUREAN,MEY,TOPAZ,ST,,CITATION HOMES,,PB,RR,PLD,0217398665,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt
4,BL,002236054,FE FENANDEZ,LORENZO,OBA,PRES S OSMENA SR,ST,523,,,PB,RR,PLD,0244072801,BULACAN OBANDO,fwso/sample txtfile.txt
5,BL,002236022,ISAGANI DE LEON,MARTIN,OBA,STO NINO,ST,46,,,PB,RR,PLD,0217426138,BULACAN OBANDO,fwso/sample txtfile.txt
6,BL,002292204,,OBANDO POLICE DEPARTMENT,OBA,J P RIZAL,ST,,,OBANDO POLICE DEPARTMENT,PB,GO,PLD,0009169555,BULACAN OBANDO,fwso/sample txtfile.txt
7,BL,002292205,,PROVINCIAL LYING-IN CLINIC,OBA,PROVINCIAL,RD,,,,PB,GO,PLD,0001347225,BULACAN OBANDO,fwso/sample txtfile.txt
8,BL,002282409,,RITE DEAL ENTERPRISES,OBA,F NAVARETTE,ST,34,,,PB,BR,PLD,0003633497,BULACAN OBANDO,fwso/sample txtfile.txt
9,BL,002292203,,RURAL BANK OF SAN PASCUAL INCORPORATED,OBA,PALIWAS,RD,,,,PB,BR,PLD,0009169490,BULACAN OBANDO,fwso/sample txtfile.txt


In [217]:
import string


update['last_name'] = update['last_name'].apply(lambda x: string.capwords(x))
update['first_name'] = update['first_name'].apply(lambda x: string.capwords(x))

update

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,list_code,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,disttribution_code,src,so_rangedate
0,BL,,,0022360409,Arleen Garcia,Cristobal,OBA,TAWIRAN,RD,45,...,PB,RR,,,,PLD,0241336611,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
1,BL,,,0022360597,,Jocelyn Forge Inc,MEY0,BAGBAGUIN,RD,56,...,PB,BR,,,,PLD,0161375209,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt,2016/07/19
2,BL,,,0022919615,,L.g Ramos Trading And Marineservices,OBA,F NAVARETTE,ST,25,...,PB,BR,,,,PLD,0212837784,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
3,BL,,,0022360493,Arriane Argamosa,Laurean,MEY0,TOPAZ,ST,,...,PB,RR,,,,PLD,0217398665,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt,2016/07/19
4,BL,,,0022360540,Fe Fenandez,Lorenzo,OBA,PRES S OSMENA SR,ST,523,...,PB,RR,,,,PLD,0244072801,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
5,BL,,,0022360228,Isagani De Leon,Martin,OBA,STO NINO,ST,46,...,PB,RR,,,,PLD,0217426138,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
6,BL,,,0022922043,,Obando Police Department,OBA,J P RIZAL,ST,,...,PB,GO,,,,PLD,0009169555,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
7,BL,,,0022922058,,Provincial Lying-in Clinic,OBA,PROVINCIAL,RD,,...,PB,GO,,,,PLD,0001347225,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
8,BL,,,0022824099,,Rite Deal Enterprises,OBA,F NAVARETTE,ST,34,...,PB,BR,,,,PLD,0003633497,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
9,BL,,,0022922033,,Rural Bank Of San Pascual Incorporated,OBA,PALIWAS,RD,,...,PB,BR,,,,PLD,0009169490,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19


In [239]:
rr=update[update.acc_type=='RR']
import string

rr['last_name'] = rr['last_name'].apply(lambda x: string.capwords(x))
rr['first_name'] = rr['first_name'].apply(lambda x: string.capwords(x))
rr['sam_stname'] = rr['sam_stname'].apply(lambda x: string.capwords(x))

AttributeError: ("'Series' object has no attribute 'split'", u'occurred at index dir_code')

In [236]:
rr_crm=pd.DataFrame()

In [273]:
rr_crm['Areacode']=rr.mem_wstd.str.slice(0,1)
rr_crm['Phone']=rr.mem_wstd.str.slice(-7)
rr_crm['name1']=rr.last_name
rr_crm['name2']=rr.first_name
rr_crm['SAM_BLDNAME']=rr.sam_bldname
rr_crm['SAM_STNMFR']=rr.sam_stnmfr
rr_crm['SAM_STNAME']=rr.sam_stname
rr_crm['SAM_STSUBT']=rr.sam_stsubt
rr_crm['sam_estate']=rr.sam_estate
rr_crm['City']=rr.distribution_code.str.split('    ',1).str.get(0)
rr_crm['Province']=rr.distribution_code.str.split('    ',1).str.get(1)
rr_crm['class_code']=""
rr_crm['class_desc']=""
rr_crm.name1 = rr_crm.name1.apply(lambda x: string.capwords(x))
rr_crm.name2 = rr_crm.name2.apply(lambda x: string.capwords(x))
rr_crm.SAM_BLDNAME = rr_crm.SAM_BLDNAME.apply(lambda x: string.capwords(x))
rr_crm.SAM_STNAME = rr_crm.SAM_STNAME.apply(lambda x: string.capwords(x))
rr_crm.SAM_STSUBT = rr_crm.SAM_STSUBT.apply(lambda x: string.capwords(x))
rr_crm.sam_estate = rr_crm.sam_estate.apply(lambda x: string.capwords(x))
rr_crm.City = rr_crm.City.apply(lambda x: string.capwords(x))
rr_crm.Province = rr_crm.Province.apply(lambda x: string.capwords(x))

In [274]:
rr_crm

,Areacode,Phone,name1,name2,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,Province,class_code,class_desc
0,0,2360409,Cristobal,Arleen Garcia,,45,Tawiran,Rd,,Bulacan,Obando,,
3,0,2360493,Laurean,Arriane Argamosa,,,Topaz,St,Citation Homes,Bulacan,Meycauayan City,,
4,0,2360540,Lorenzo,Fe Fenandez,,523,Pres S Osmena Sr,St,,Bulacan,Obando,,
5,0,2360228,Martin,Isagani De Leon,,46,Sto Nino,St,,Bulacan,Obando,,


In [225]:
import pandas as pd
import time
so_file='fwso/sample txtfile.txt'
widths = [4,2,15,10,30,100,4,30,3,7,1,30,30,2,20,2,2,10,9,50,3,30,50]
update=pd.read_fwf(so_file,
                   header=None, 
                   widths=widths,
                   names=['dir_code','so_type','sonumber','mem_wstd','first_name','last_name','sam_city'
                          ,'sam_stname','sam_stsubt','sam_stnmfr','sam_subd','sam_estate','sam_bldname','sam_aptcod','sam_aprmnt'
                          ,'list_code','acc_type','old_wstd','so_date','joint_user','regulatory_id','account_no','distribution_code'],
                   converters = {'mem_wstd': str, 'sam_stnmfr':str,'account_no':str},
                   index_col=None
                  )
#update.()
update = update.fillna('')

update.sort_values(by='last_name',inplace=True)
update.reset_index(drop=True,inplace=True)
update['acc_type']=update['acc_type'].astype('category')
update['src']=so_file
update['so_rangedate']=time.strftime("%Y/%m/%d")
update

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,list_code,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,src,so_rangedate
0,BL,,,0022360409,ARLEEN GARCIA,CRISTOBAL,OBA,TAWIRAN,RD,45,...,PB,RR,,,,PLD,0241336611,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
1,BL,,,0022360597,,JOCELYN FORGE INC,MEY0,BAGBAGUIN,RD,56,...,PB,BR,,,,PLD,0161375209,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt,2016/07/19
2,BL,,,0022919615,,L.G RAMOS TRADING AND MARINESERVICES,OBA,F NAVARETTE,ST,25,...,PB,BR,,,,PLD,0212837784,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
3,BL,,,0022360493,ARRIANE ARGAMOSA,LAUREAN,MEY0,TOPAZ,ST,,...,PB,RR,,,,PLD,0217398665,BULACAN MEYCAUAYAN CITY,fwso/sample txtfile.txt,2016/07/19
4,BL,,,0022360540,FE FENANDEZ,LORENZO,OBA,PRES S OSMENA SR,ST,523,...,PB,RR,,,,PLD,0244072801,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
5,BL,,,0022360228,ISAGANI DE LEON,MARTIN,OBA,STO NINO,ST,46,...,PB,RR,,,,PLD,0217426138,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
6,BL,,,0022922043,,OBANDO POLICE DEPARTMENT,OBA,J P RIZAL,ST,,...,PB,GO,,,,PLD,0009169555,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
7,BL,,,0022922058,,PROVINCIAL LYING-IN CLINIC,OBA,PROVINCIAL,RD,,...,PB,GO,,,,PLD,0001347225,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
8,BL,,,0022824099,,RITE DEAL ENTERPRISES,OBA,F NAVARETTE,ST,34,...,PB,BR,,,,PLD,0003633497,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
9,BL,,,0022922033,,RURAL BANK OF SAN PASCUAL INCORPORATED,OBA,PALIWAS,RD,,...,PB,BR,,,,PLD,0009169490,BULACAN OBANDO,fwso/sample txtfile.txt,2016/07/19
